In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
train=pd.read_excel("Final_Train.xlsx")
test=pd.read_excel("Final_Test.xlsx")

In [3]:
frames = [train, test]
df=pd.concat(frames,sort=False)

In [4]:
df.head()

,Qualification,Experience,Rating,Place,Profile,Miscellaneous_Info,Fees
0,"BHMS, MD - Homeopathy",24 years experience,100%,"Kakkanad, Ernakulam",Homeopath,"100% 16 Feedback Kakkanad, Ernakulam",100.0
1,"BAMS, MD - Ayurveda Medicine",12 years experience,98%,"Whitefield, Bangalore",Ayurveda,"98% 76 Feedback Whitefield, Bangalore",350.0
2,"MBBS, MS - Otorhinolaryngology",9 years experience,NaN,"Mathikere - BEL, Bangalore",ENT Specialist,NaN,300.0
3,"BSc - Zoology, BAMS",12 years experience,NaN,"Bannerghatta Road, Bangalore",Ayurveda,"Bannerghatta Road, Bangalore ₹250 Available on...",250.0
4,BAMS,20 years experience,100%,"Keelkattalai, Chennai",Ayurveda,"100% 4 Feedback Keelkattalai, Chennai",250.0


In [5]:
df.isnull().sum()

Qualification            0
Experience               0
Rating                4392
Place                   31
Profile                  0
Miscellaneous_Info    3454
Fees                  1987
dtype: int64

In [6]:
df["Experience"] = df["Experience"].str.split()
df["Experience"] = df["Experience"].str[0].astype("int")

In [7]:
df["Place"].fillna("Unknown,Unknown",inplace=True)
df["Place"] = df["Place"].str.split(",")
df["City"] = df["Place"].str[-1]
df["Place"] = df["Place"].str[0]

In [8]:
df["Rating"].value_counts()

100%    907
98%     402
99%     327
97%     325
96%     285
95%     225
94%     166
93%     143
92%      95
90%      95
91%      86
88%      54
89%      53
85%      35
83%      32
82%      32
86%      27
80%      26
87%      25
84%      20
67%      17
77%      17
71%      16
79%      15
81%      15
73%      13
76%      13
60%      11
70%       8
75%       8
74%       7
78%       7
69%       6
55%       5
68%       5
56%       5
64%       4
57%       3
72%       3
53%       2
65%       2
62%       2
63%       2
33%       2
58%       1
47%       1
52%       1
45%       1
7%        1
48%       1
40%       1
36%       1
Name: Rating, dtype: int64

In [9]:
df["Rating"].fillna("-99%",inplace=True)
df["Rating"] = df["Rating"].str[:-1].astype("int")
bins = [-99,0,10,20,30,40,50,60,70,80,90,100]
labels = [i for i in range(11)]
df["Rating"] = pd.cut(df["Rating"],bins=bins,labels=labels,include_lowest=True)

In [10]:
df["Rating"].value_counts()

0     4392
10    2961
9      388
8      125
7       46
6       28
4        4
5        3
1        1
3        0
2        0
Name: Rating, dtype: int64

In [11]:
df["Qualification"].value_counts()

BDS                                                                                                                                                                                                                                                                        632
BHMS                                                                                                                                                                                                                                                                       627
BAMS                                                                                                                                                                                                                                                                       624
MBBS                                                                                                                                                                                       

In [12]:
df["Qualification"]=df["Qualification"].str.split(",")
Qualification ={}
for x in df["Qualification"].values:
    for each in x:
        each = each.strip()
        if each in Qualification:
            Qualification[each]+=1
        else:
            Qualification[each]=1

In [13]:
most_qua = sorted(Qualification.items(),key=lambda x:x[1],reverse=True)[:10]
final_qua =[]
for tup in most_qua:
    final_qua.append(tup[0])
for title in final_qua:
    df[title]=0
    
for x,y in zip(df["Qualification"].values,np.array([idx for idx in range(len(df))])):
    for q in x:
        q = q.strip()
        if q in final_qua:
            df[q][y] = 1
df.drop("Qualification",axis=1,inplace=True)

In [14]:
df["Profile"].value_counts()

Dentist             1838
General Medicine    1617
Dermatologists      1373
Homeopath           1120
Ayurveda            1037
ENT Specialist       963
Name: Profile, dtype: int64

In [15]:
 df["City"].value_counts()

 Bangalore             1678
 Mumbai                1608
 Delhi                 1602
 Hyderabad             1284
 Chennai               1142
 Coimbatore             298
 Ernakulam              201
 Thiruvananthapuram     103
Unknown                  31
e                         1
Name: City, dtype: int64

In [16]:
df.head()

,Experience,Rating,Place,Profile,Miscellaneous_Info,Fees,City,MBBS,BDS,BAMS,BHMS,MD - Dermatology,MS - ENT,Venereology & Leprosy,MD - General Medicine,Diploma in Otorhinolaryngology (DLO),MD - Homeopathy
0,24,10,Kakkanad,Homeopath,"100% 16 Feedback Kakkanad, Ernakulam",100.0,Ernakulam,0,0,0,1,0,0,0,0,0,1
1,12,10,Whitefield,Ayurveda,"98% 76 Feedback Whitefield, Bangalore",350.0,Bangalore,0,0,1,0,0,0,0,0,0,0
2,9,0,Mathikere - BEL,ENT Specialist,NaN,300.0,Bangalore,1,0,0,0,0,0,0,0,0,0
3,12,0,Bannerghatta Road,Ayurveda,"Bannerghatta Road, Bangalore ₹250 Available on...",250.0,Bangalore,0,0,1,0,0,0,0,0,0,0
4,20,10,Keelkattalai,Ayurveda,"100% 4 Feedback Keelkattalai, Chennai",250.0,Chennai,0,0,1,0,0,0,0,0,0,0


In [17]:
df=df.drop(["Miscellaneous_Info"],axis=1)

In [18]:
df.isnull().sum()

Experience                                 0
Rating                                     0
Place                                      0
Profile                                    0
Fees                                    1987
City                                       0
MBBS                                       0
BDS                                        0
BAMS                                       0
BHMS                                       0
MD - Dermatology                           0
MS - ENT                                   0
Venereology & Leprosy                      0
MD - General Medicine                      0
Diploma in Otorhinolaryngology (DLO)       0
MD - Homeopathy                            0
dtype: int64

In [19]:
df["Fees"]=df["Fees"].fillna(df["Fees"].mean())

In [20]:
df.isnull().sum()

Experience                              0
Rating                                  0
Place                                   0
Profile                                 0
Fees                                    0
City                                    0
MBBS                                    0
BDS                                     0
BAMS                                    0
BHMS                                    0
MD - Dermatology                        0
MS - ENT                                0
Venereology & Leprosy                   0
MD - General Medicine                   0
Diploma in Otorhinolaryngology (DLO)    0
MD - Homeopathy                         0
dtype: int64

In [21]:
df = pd.get_dummies(df,columns=["City","Profile"],prefix=["City","Profile"])

In [22]:
df.head()

,Experience,Rating,Place,Fees,MBBS,BDS,BAMS,BHMS,MD - Dermatology,MS - ENT,...,City_ Mumbai,City_ Thiruvananthapuram,City_Unknown,City_e,Profile_Ayurveda,Profile_Dentist,Profile_Dermatologists,Profile_ENT Specialist,Profile_General Medicine,Profile_Homeopath
0,24,10,Kakkanad,100.0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,12,10,Whitefield,350.0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,9,0,Mathikere - BEL,300.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,12,0,Bannerghatta Road,250.0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,20,10,Keelkattalai,250.0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [27]:
df.head()

,Experience,Rating,Place,Fees,MBBS,BDS,BAMS,BHMS,MD - Dermatology,MS - ENT,...,City_ Mumbai,City_ Thiruvananthapuram,City_Unknown,City_e,Profile_Ayurveda,Profile_Dentist,Profile_Dermatologists,Profile_ENT Specialist,Profile_General Medicine,Profile_Homeopath
0,24,10,Kakkanad,100.0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,12,10,Whitefield,350.0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,9,0,Mathikere - BEL,300.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,12,0,Bannerghatta Road,250.0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,20,10,Keelkattalai,250.0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [48]:
df.dtypes
df.skew()

Experience                               0.984224
Rating                                   0.235010
Fees                                     0.868130
MBBS                                     0.126091
BDS                                      1.283447
BAMS                                     2.221212
BHMS                                     2.258682
MD - Dermatology                         2.676054
MS - ENT                                 3.407118
Venereology & Leprosy                    4.188374
MD - General Medicine                    4.265539
Diploma in Otorhinolaryngology (DLO)     4.653745
MD - Homeopathy                          5.675347
City_ Bangalore                          1.415970
City_ Chennai                            2.032012
City_ Coimbatore                         4.870220
City_ Delhi                              1.488145
City_ Ernakulam                          6.048309
City_ Hyderabad                          1.839561
City_ Mumbai                             1.482310


In [89]:
x=df.drop(["Fees","Place","City_e"],axis=1)
from sklearn.preprocessing import StandardScaler,OrdinalEncoder

enc = OrdinalEncoder()
x = enc.fit_transform(x)

scaler = StandardScaler()
x = scaler.fit_transform(x)
x




array([[ 0.58525227,  1.17083142, -0.9389509 , ..., -0.37130427,
        -0.50538093,  2.46909469],
       [-0.4886861 ,  1.17083142, -0.9389509 , ..., -0.37130427,
        -0.50538093, -0.40500674],
       [-0.75717069, -0.89310384,  1.06501842, ...,  2.69320903,
        -0.50538093, -0.40500674],
       ...,
       [-0.75717069, -0.89310384, -0.9389509 , ..., -0.37130427,
         1.97870544, -0.40500674],
       [-0.4886861 ,  1.17083142,  1.06501842, ..., -0.37130427,
        -0.50538093, -0.40500674],
       [-0.84666555, -0.89310384,  1.06501842, ..., -0.37130427,
        -0.50538093, -0.40500674]])

In [76]:
y=df["Fees"].values.reshape(-1,1)


In [83]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.2,random_state=45)

In [84]:
from sklearn.metrics import mean_squared_error
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(6358, 27)
(1590, 27)
(6358, 1)
(1590, 1)


In [85]:
lm=LinearRegression()
lm.fit(x_train,y_train)
lm.coef_

array([[-8.61275292e+00,  1.82254872e+01,  1.38767298e+01,
        -7.00545862e+00, -5.09285770e+00, -1.71538108e-01,
         3.80041571e+00,  6.44669040e+00,  1.04398750e+00,
         3.55261849e+00, -9.35324916e+00,  2.12096407e+00,
         8.16354123e+14,  7.01661854e+14,  3.80003583e+14,
         8.02472451e+14,  3.14060739e+14,  7.36205145e+14,
         8.03593644e+14,  2.26237154e+14,  1.24683795e+14,
        -1.02563899e+14, -1.28389071e+14, -1.15111270e+14,
        -9.93644534e+13, -1.22581764e+14, -1.05947440e+14]])

In [86]:
lm.score(x_train,y_train)

0.14763698797275993

In [87]:
def linreg():
    lm.fit(x_train,y_train)
    pred=lm.predict(x_test)
    print(lm.score(x_train,y_train))
    print(r2_score(y_test,pred))
    print(mean_squared_error(y_test,pred))

In [88]:
linreg()

0.14763698797275993
-8.563283894325488e+22
2.516612078780311e+27


In [54]:
lm=LinearRegression()
lm.fit(x_train,y_train)
lm.coef_

array([[-9.29309853e+00,  1.78363293e+01,  1.65031656e+01,
        -5.52422071e+00, -4.69310817e+00,  4.04324713e-01,
         3.25029687e+00,  6.15446460e+00, -3.17233247e-01,
         3.27336506e+00, -1.04042020e+01,  2.88147715e+00,
        -4.38632163e+14, -3.77007291e+14, -2.04178296e+14,
        -4.31173454e+14, -1.68746795e+14, -3.95567617e+14,
        -4.31775878e+14, -1.21558634e+14, -6.69933808e+13,
         5.95983371e+13,  7.46049557e+13,  6.68894254e+13,
         5.77391873e+13,  7.12304166e+13,  6.15644616e+13]])

In [70]:
lm.score(x_train,y_train)

0.15210806095434193

In [71]:
from sklearn.svm import SVR
m = SVR(gamma="scale")
m.fit(x_train,y_train)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [72]:
def score(y_pred,y):
    y_pred = np.log(y_pred)
    y = np.log(y)
    return 1-((np.sum((y_pred-y)**2))/len(y))**1/2

In [73]:
y_pred = m.predict(scaler.transform(x_test))
score(y_pred,y_test)

-464.23299878234127